<a href="https://colab.research.google.com/github/ChingizIbnVaxob/data-science-practices/blob/main/25.03.2025/Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sklearn

In [4]:
URL = "https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df = pd.read_csv(URL)

Ma'lumotlarni train va testga ajratamiz

In [7]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

X_train = train_set.drop("median_house_value", axis=1)
y = train_set["median_house_value"].copy()

X_num = X_train.drop("ocean_proximity", axis=1)

In [8]:
from sklearn.linear_model import LinearRegression

LR_model = LinearRegression( )

Pipeline quramiz

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
# bizga kerak ustunlar indekslari
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self # bizni funksiyamiz faqat transformer. estimator emas
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room: # add_bedrooms_per_room ustuni ixtiyoriy bo'ladi
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipeline = Pipeline([
          ('imputer', SimpleImputer(strategy='median')),
          ('attribs_adder', CombinedAttributesAdder(add_bedrooms_per_room = True)),
          ('std_scaler', StandardScaler())
])

Matnli ustunlar uchun

In [12]:
from sklearn.compose import ColumnTransformer

num_attribs = list(X_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

In [13]:
X_prepared = full_pipeline.fit_transform(X_train)

In [14]:
X_prepared

array([[ 1.27258656, -1.3728112 ,  0.34849025, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.70916212, -0.87669601,  1.61811813, ...,  0.        ,
         0.        ,  1.        ],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ...,  0.        ,
         0.        ,  0.        ],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.        ,
         0.        ,  0.        ],
       [-1.41489815,  0.99543676,  1.85617335, ...,  0.        ,
         1.        ,  0.        ]])

#  Linear regression

In [19]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()

In [21]:
LR_model.fit(X_prepared, y)

LinearRegression()

In [24]:
test_data = X_train.sample(10)
test_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
2394,-119.40,37.09,22.0,2211.0,477.0,773.0,288.0,3.3269,INLAND
19105,-122.65,38.27,9.0,4764.0,816.0,2077.0,755.0,5.1391,<1H OCEAN
956,-121.92,37.69,13.0,3742.0,555.0,1590.0,559.0,7.3160,<1H OCEAN
16843,-122.44,37.63,35.0,5113.0,959.0,3004.0,964.0,4.7625,NEAR OCEAN
14143,-117.09,32.74,42.0,1986.0,472.0,1472.0,475.0,2.1757,NEAR OCEAN
17925,-121.98,37.34,18.0,6649.0,1712.0,3604.0,1651.0,4.5368,<1H OCEAN
1064,-121.85,39.75,39.0,568.0,127.0,267.0,129.0,1.8095,INLAND
11168,-117.99,33.82,33.0,2342.0,475.0,1367.0,509.0,4.1167,<1H OCEAN
14106,-117.10,32.75,23.0,1858.0,551.0,1506.0,492.0,1.7446,NEAR OCEAN
6438,-118.02,34.10,36.0,452.0,80.0,248.0,83.0,1.9688,INLAND


In [27]:
test_label = y.loc[test_data.index]
test_label

2394     102700.0
19105    234500.0
956      285400.0
16843    281300.0
14143    110100.0
17925    307400.0
1064      78100.0
11168    215500.0
14106     85200.0
6438     226000.0
Name: median_house_value, dtype: float64

In [25]:
X_train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596,NEAR OCEAN
8267,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125,NEAR OCEAN
17445,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563,NEAR OCEAN
14265,-117.11,32.69,36.0,1421.0,367.0,1418.0,355.0,1.9425,NEAR OCEAN
2271,-119.80,36.78,43.0,2382.0,431.0,874.0,380.0,3.5542,INLAND
...,...,...,...,...,...,...,...,...,...
11284,-117.96,33.78,35.0,1330.0,201.0,658.0,217.0,6.3700,<1H OCEAN
11964,-117.43,34.02,33.0,3084.0,570.0,1753.0,449.0,3.0500,INLAND
5390,-118.38,34.03,36.0,2101.0,569.0,1756.0,527.0,2.9344,<1H OCEAN
860,-121.96,37.58,15.0,3575.0,597.0,1777.0,559.0,5.7192,<1H OCEAN


In [29]:
test_data_prepared = full_pipeline.transform(test_data)
predicted_labels = LR_model.predict(test_data_prepared)

In [31]:
predicted_labels

array([120010.09232116, 265504.79241275, 342738.51745966, 281669.76051448,
       168874.88142414, 322941.04518877,  61039.51664022, 232939.17670982,
       146562.10619407,  91505.97400674])

In [33]:
pd.DataFrame({'Bashorat':predicted_labels, 'Asl qiymat': test_label})

,Bashorat,Asl qiymat
2394,120010.092321,102700.0
19105,265504.792413,234500.0
956,342738.517460,285400.0
16843,281669.760514,281300.0
14143,168874.881424,110100.0
17925,322941.045189,307400.0
1064,61039.516640,78100.0
11168,232939.176710,215500.0
14106,146562.106194,85200.0
6438,91505.974007,226000.0
